1) Подсчитать количество слов с отзывами == 5.0

In [1]:
import dask.bag as db
import re
import dask

import json
import gzip

In [2]:
def overall_5(x):
    return x.get('overall') == 5.0

In [13]:
b = db.read_text('../../data/amazon-reviews/reviews_Video_Games_5.json.gz').map(json.loads)

In [14]:
b = b.filter(overall_5).map(lambda x: re.sub('''[!@#$.?:;\"(),]''', "", x.get('reviewText').lower().strip()).split()).flatten().frequencies(sort = True)

In [15]:
b = b.compute()

In [16]:
len(b)

360064

In [17]:
s = 0
for i in b:
    s += i[1]
print(s)

20746804


2) Для одного из файлов с отзывами из файлов. Подсчитать частоту (!) слов для отзывов с оценкой "5" и для отзывов с оценкой "2" и найти 50 распространенных слов, частота которых максимально увеличивается и уменьшается при переходе от плохих отзывов к хорошим.

In [2]:
def overall_5(x):
    return x.get('overall') == 5.0

In [3]:
def overall_2(x):
    return x.get('overall') == 2.0

In [4]:
b5 = db.read_text('../../data/amazon-reviews/reviews_Video_Games_5.json.gz').map(json.loads)

In [5]:
b5 = b5.filter(overall_5).map(lambda x: re.sub('''[!@#$.?:;\"(),]''', "", x.get('reviewText').lower().strip())\
                            .split()).flatten().frequencies(sort = True)

In [6]:
b2 = db.read_text('../../data/amazon-reviews/reviews_Video_Games_5.json.gz').map(json.loads)

In [7]:
b2 = b2.filter(overall_2).map(lambda x: re.sub('''[!@#$.?:;\"(),]''', "", x.get('reviewText').lower().strip())\
                            .split()).flatten().frequencies(sort = True)

In [8]:
b5, b2 = dask.compute(b5, b2)

In [9]:
s5 = 0
for i in b5:
    s5 += i[1]
print(s5)
s2 = 0
for i in b2:
    s2 += i[1]
print(s2)

20746804
3329225


In [70]:
b5_1 = db.from_sequence(b5) 
b5_1 = b5_1.map(lambda x: (x[0],x[1]/s5))

b2_1 = db.from_sequence(b2) 
b2_1 = b2_1.map(lambda x: (x[0],x[1]/s2))

In [71]:
b5_1, b2_1 = dask.compute(b5_1, b2_1)

In [72]:
b2_1 = dict(b2_1)

In [73]:
b5_1 = dict(b5_1)

In [86]:
help_dict = {}
for k, v in b5_1.items():
    try:
        help_dict[k] = abs(b5_1[k] - b2_1.get(k))
    except:
        continue

In [87]:
help_dict

{'the': 0.00015708556478240265,
 'and': 0.0031782218262378682,
 'to': 0.001814063976402415,
 'a': 0.0008994066744856186,
 'of': 0.0008666619631825002,
 'is': 0.002074720262718561,
 'you': 0.0017347658239966063,
 'i': 0.00113253584370537,
 'game': 7.423364228504106e-05,
 'it': 0.0004047351303419558,
 'this': 0.0008182856380665009,
 'in': 6.96178862633623e-05,
 'that': 0.0009946013180574358,
 'for': 0.0005451221723153706,
 'are': 0.00031069142698932593,
 'with': 0.0007522832661965434,
 'but': 0.0010926680863855304,
 'on': 0.0002370304905262256,
 'have': 5.174948039098765e-07,
 'as': 0.0009510448364266812,
 'can': 0.0017526062716638233,
 'not': 0.0017179441238443323,
 'your': 0.0005770395965630651,
 'was': 0.0015632173440565042,
 'be': 0.00039218954937081385,
 'games': 0.0012981782747405325,
 'like': 0.0003366589999736387,
 'my': 0.0006642460058828912,
 'all': 2.298207448063038e-05,
 'so': 1.6211709098366313e-05,
 'if': 1.9105591796738514e-05,
 'one': 0.000610769594240999,
 'or': 0.000640

In [89]:
sorted(help_dict.items(), key=lambda x: x[1])[-51:-1]

[('could', 0.0004618744403081847),
 ('nothing', 0.00046433676309617103),
 ('why', 0.0004707692446947144),
 ('after', 0.00047457059240678216),
 ('because', 0.00047613093772599447),
 ('awesome', 0.00048245584209497254),
 ('only', 0.0005017641284362716),
 ('amazing', 0.0005021685853511597),
 ("don't", 0.0005076218381148336),
 ('me', 0.0005116852412196562),
 ('were', 0.0005225425172599256),
 ('for', 0.0005451221723153706),
 ('your', 0.0005770395965630651),
 ('even', 0.0005881541107491),
 ('well', 0.0005995749084524932),
 ('new', 0.0006074725778790744),
 ('one', 0.000610769594240999),
 ('boring', 0.0006110296783370222),
 ('same', 0.0006296009476347761),
 ('play', 0.0006319755050011541),
 ('or', 0.0006403768672734204),
 ('fun', 0.0006513913038268381),
 ('my', 0.0006642460058828912),
 ('will', 0.0006857708473755999),
 ('at', 0.000692256720148784),
 ('would', 0.0007241870806629989),
 ('also', 0.0007300960327493414),
 ('has', 0.0007323431267920789),
 ('with', 0.0007522832661965434),
 ('bad', 0.